In [45]:
import numpy as np
from collections import Counter
def entropy(y):
    hist = np.bincount(y)
    ps = hist/len(y)
    return -np.sum([p*np.log2(p) for p in ps if p > 0])
class Node:
    def __init__(self, feature=None, threshold = None, left = None, right=None,*,value = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    def is_leaf(self):
        return self.value is not None

class DecisionTree:
    def __init__(self,min_samples_split = 2,max_depth = 100,n_feats = None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None
    def most_common_labels(self,y):
        counter = Counter(y)
        most_comm = counter.most_common(1)[0][0]
        return most_comm

    def best_criteria(self,x,y,feat_ids):
        best_gain = -1
        split_ids, split_thresh = None, None
        for id1 in feat_ids:
            x_col = x[:,id1]
            thresholds = np.unique(x_col)
            for threshold in thresholds:
                gain = self.information_gain(y, x_col, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_id = id1
                    split_thresh = threshold
        return split_id, split_thresh
    def information_gain(self,y,x_col, threshold):
        parent = entropy(y)
        left_ids, right_ids = self.split(x_col, threshold)
        if len(left_ids) == 0 or len(right_ids) == 0:
            return 0
        n = len(y)
        n_l, n_r = len(left_ids),len(right_ids)
        e_l, e_r = entropy(y[left_ids]),entropy(y[right_ids])
        child = (n_l/n) * e_l + (n_r/n) * e_r
        ig = parent - child
        return ig
        
        
        
        
    def split(self, x_col, split_thresh):
        left_ids = np.argwhere(x_col <= split_thresh).flatten()
        right_ids = np.argwhere(x_col > split_thresh).flatten()
        return left_ids, right_ids
        
    def grow_tree(self,x,y,depth = 0):
        n_samples, n_features = x.shape
        n_labels = len(np.unique(y))
        
        if (depth > self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self.most_common_labels(y)
            return Node(value = leaf_value)
        feat_ids = np.random.choice(n_features,self.n_feats,replace = False)
        best_feat, best_thresh = self.best_criteria(x,y,feat_ids)
        left_ids, right_ids = self.split(x[:,best_feat],best_thresh)
        left = self.grow_tree(x[left_ids,:], y[left_ids],depth+1)
        right = self.grow_tree(x[right_ids,:],y[right_ids],depth+1)
        return Node(best_feat, best_thresh, left, right)
        
    def fit(self,x,y):
        self.n_feats = x.shape[1] if not self.n_feats else min(self.n_feats, x.shape[1])
        
        self.root = self.grow_tree(x,y)
    
    def predict(self,xs):
            
        return np.array([self.traverse_tree(x,self.root) for x in xs])
    def traverse_tree(self,x, node):
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self.traverse_tree(x,node.left)
        return self.traverse_tree(x,node.right)
    
        
        

In [46]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
def accuracy(y,yt):
    return np.sum(y == yt) / len(y)
data = datasets.load_breast_cancer()
x = data.data
y = data.target
xt,xtest,yt,ytest = train_test_split(x,y,test_size=0.2,random_state=1234)
clf = DecisionTree(max_depth=20)

In [47]:
clf.fit(xt,yt)

In [48]:
ypred = clf.predict(xtest)
acc = accuracy(ytest,ypred)
print(acc)

0.9210526315789473


In [53]:
def bootstrap(x,y):
    n_samples = x.shape[0]
    ids = np.random.choice(n_samples, size = n_samples, replace=True)
    return x[ids], y[ids]
class RandomForest:
    
    def __init__(self, n_trees=100,min_sample_split=2,max_depth = 100, n_feats = None):
        self.n_trees = n_trees
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []
    
    def most_common_labels(self,y):
        counter = Counter(y)
        most_comm = counter.most_common(1)[0][0]
        return most_comm
    
    def fit(self,x,y):
        self.trees = []
        for i in range(self.n_trees):
            tree = DecisionTree(min_samples_split=self.min_sample_split, max_depth=self.max_depth, n_feats=self.n_feats)   
            xsample,ysample = bootstrap(x,y)
            tree.fit(xsample,ysample)
            self.trees.append(tree)
    
    def predict(self,x):
        tree_preds = np.array([tree.predict(x) for tree in self.trees])
        # 
        tree_preds = np.swapaxes(tree_preds,0,1)
        y_pred = np.array([self.most_common_labels(tree_pred) for tree_pred in tree_preds])
        return y_pred
    

In [57]:
rf = RandomForest(n_trees = 5)
rf.fit(xt,yt)

In [58]:
preds = rf.predict(xtest)
acc = accuracy(preds, ytest)


In [59]:
acc

0.9298245614035088